<a href="https://colab.research.google.com/github/danielsippel/colab/blob/main/assets/Applio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Applio**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio) — [Terms of Use](https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md)

<br>

---

<br>

#### **Acknowledgments**

To all external collaborators for their special help in the following areas: Hina (Encryption method), Poopmaster (Extra section), Shirou (UV installer)

#### **Disclaimer**
By using Applio, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio disclaims liability and reserves the right to amend these terms.

In [1]:
# @title **Install Applio**
import sys
from IPython.display import clear_output
import codecs
import os

encoded_url = "uggcf://tvguho.pbz/VNUvfcnab/Nccyvb/"
decoded_url = codecs.decode(encoded_url, "rot_13")

repo_name_encoded = "Nccyvb"
repo_name = codecs.decode(repo_name_encoded, "rot_13")

!pip install -q uv

!git clone --depth 1 {decoded_url} --branch 3.2.9 --single-branch
%cd {repo_name}
clear_output()

print("Installing requirements...")
!uv pip install torch==2.7.1 torchvision torchaudio==2.7.1 \
  --index-url https://download.pytorch.org/whl/cu128 \
  -q
!uv pip install -r requirements.txt -q
!uv pip install pyngrok -q

clear_output()
print("✅ Finished installing requirements!")

✅ Finished installing requirements!


In [ ]:
# @title **Start Applio**
# @markdown  ### Choose a sharing method:

import codecs
import threading
import urllib.request
import time
import ipywidgets as widgets
from IPython.display import display
import os
from pyngrok import ngrok
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = "If you selected the 'ngrok' method, obtain your auth token here: https://dashboard.ngrok.com/get-started/your-authtoken" # @param {type:"string"}
tensorboard = False #@param {type: "boolean"}

def start_gradio():
    !python app.py --listen --share

def start_localtunnel():
    !npm install -g localtunnel &>/dev/null
    with open('url.txt', 'w') as file:
        file.write('')
    get_ipython().system_raw('lt --port 6969 >> url.txt 2>&1 &')
    time.sleep(2)
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        tunnel_url = file.read()
        tunnel_url = tunnel_url.replace("your url is: ", "")

    print(f"Share Link: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m\n")

    password_endpoint_widget = widgets.Text(
    value=endpoint_ip,
    description='Password IP:',
    disabled=True
    )
    display(password_endpoint_widget)
    !python app.py --listen


def start_ngrok():
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(6969)
        print(f"Ngrok URL: \033[0m\033[93m{tunnel.public_url}\033[0m", end="\033[0m\n")
        !python app.py --listen
    except Exception as e:
        print(f"Error starting ngrok: {e}")

def start_applio():
    if method == 'gradio':
        start_gradio()
    elif method == 'localtunnel':
        start_localtunnel()
    elif method == 'ngrok':
        start_ngrok()

if tensorboard:
    %load_ext tensorboard
    %reload_ext tensorboard
    %tensorboard --logdir logs --bind_all

if "autobackups" not in globals():
    autobackups = False

if autobackups:
    thread = threading.Thread(target=backup_files)
    thread.start()

thread_applio = threading.Thread(target=start_applio)
thread_applio.start()


while True:
    time.sleep(5)


Starting backup loop...
Files are up to date.
An error occurred connecting to Discord: Could not find Discord installed and running on this machine.
* Running on local URL:  http://127.0.0.1:6969
* Running on public URL: https://5c62ca7f0c239f17c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Starting preprocess with 12 processes...
100% 19/19 [00:03<00:00,  5.92it/s]
Preprocess completed in 3.21 seconds on 00:33:19 seconds of audio.
Starting pitch extraction with 12 cores on cuda:0 using rmvpe...
Backup Complete: 1232 new, 1 updated, 0 deleted.
Backup Complete: 0 new, 1 updated, 0 deleted.
  0% 0/616 [00:00<?, ?it/s]Files are up to date.
 18% 112/616 [00:04<00:16, 31.47it/s]Backup Complete: 1393 new, 1 updated, 0 deleted.
 31% 189/616 [00:06<00:14, 28.68it/s]Backup Complete: 163 new, 1 updated, 0 deleted.
 42%

### **Extra**
Enjoy extra options that can make it easier for you to use Applio


In [7]:
!ln -s "/content/drive/MyDrive/voicemodel-source_rode_nt1_wav" "assets/datasets/voicemodel-source_rode_nt1_wav"
!ln -s "/content/drive/MyDrive/voicemodel-source_metaglasses_wav" "assets/datasets/voicemodel-source_metaglasses_wav"
epochs = 150

In [4]:
# @title Mount Drive
# @markdown Mount the files from Google Drive to the Colab.
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# @title Auto Backup
# @markdown When running it, it will be activated or deactivated previously to start up together with Applio.
import os
import shutil
import time

LOGS_FOLDER = "/content/Applio/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/ApplioBackup"

if "autobackups" not in globals():
    autobackups = False

cooldown = 15  # @param {type:"slider", min:0, max:100, step:0}
def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" and "mute" directories
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(current_timestamp)
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1


            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]


            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)


        except Exception as error:
            print(f"An error occurred during backup: {error}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

Autobackup Enabled


In [5]:
# @title Load a Backup
import os
import shutil

# @markdown Put the exact name you put as your Model Name in Applio.
modelname = "zwanglos-v2-rode_nt1"  # @param {type:"string"}
source_path = "/content/drive/MyDrive/ApplioBackup/" + modelname
destination_path = "/content/Applio/logs/" + modelname
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/ApplioBackup/", backup_timestamps_file)
    time__ = os.path.join("/content/Applio/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

Model backup loaded successfully.
